In [1]:
from copy import deepcopy
from pathlib import Path
import pickle
from typing import Literal

from gears import PertData
import seaborn as sns
from sklearn.linear_model import ElasticNet
import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

/home/users/geroldcsendes/miniconda3/envs/scgpt_2_clone/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def eval(pred: np.ndarray, data: pd.DataFrame, de_target: bool):
    ctrl = data['ctrl_mean']
    assert pred.shape[1] == ctrl.shape[0]

    corrs = []
    for i in range(pred.shape[0]):
        _pred = pred[i]
        _target = data['test_y'].iloc[i].values
        if not de_target:
            _pred = _pred - ctrl
            _target = _target - ctrl
        r = pearsonr(_pred, _target)[0]
        corrs.append(r)

    print(f"Mean correlation: {np.mean(corrs)}")
    return corrs

In [3]:
def _get_go_features(is_norman: bool, perturbation_list: list, go_pca: pd.DataFrame):

    GO_NULL_STR = "NULL"
    go_pca = go_pca.copy()
    pert_list = [i.replace("ctrl+", "").replace("+ctrl", "") for i in perturbation_list]
    if is_norman:
        # insert NULL as a row to the go_pca with 0 values
        go_pca.loc[GO_NULL_STR] = 0

        pert1 = []
        pert2 = []
        for i in pert_list:
            if '+' in i:
                p1, p2 = i.split('+')
                pert1.append(p1)
                pert2.append(p2)
            else:
                pert1.append(i)
                pert2.append('NULL')
        
        go_features = go_pca.loc[pert1].values - go_pca.loc[pert2].values

    else:
        go_features = go_pca.loc[pert_list]
    return go_features

def create_bulk_data(pert_data, go_pca_, is_norman: bool, de_target: bool):

    data_splits = deepcopy(pert_data.set2conditions)

    train_samples = data_splits['train'] + data_splits['val']
    train_samples.remove('ctrl')
    test_samples = data_splits['test']

    assert 'ctrl' not in test_samples, "ctrl in test samples"

    data = pert_data.adata.to_df()
    data['condition'] = pert_data.adata.obs['condition']
    data = data.groupby('condition').mean()

    train_x = _get_go_features(is_norman, train_samples, go_pca_)
    train_y = data.loc[train_samples].copy()
    test_x = _get_go_features(is_norman, test_samples, go_pca_)
    test_y = data.loc[test_samples].copy()
    ctrl_mean = data.loc['ctrl'].values

    if de_target:
        train_y = train_y - ctrl_mean
        test_y = test_y - ctrl_mean

    assert train_x.shape[0] == train_y.shape[0], "Train shapes do not match"
    assert test_x.shape[0] == test_y.shape[0], "Test shapes do not match"
    assert train_x.shape[1] == test_x.shape[1], "Train and test features do not match"
    assert train_y.shape[1] == test_y.shape[1], "Train and test targets do not match"
    assert ctrl_mean.shape[0] == train_y.shape[1] == test_y.shape[1], "ctrl mean shape does not match"
    assert ctrl_mean.ndim == 1, "ctrl mean is not 1D"

    print("Number of training samples: ", train_x.shape[0])
    print("Number of test samples: ", test_x.shape[0])

    return dict(
        train_x=train_x,
        train_y=train_y,
        test_x=test_x,
        test_y=test_y,
        ctrl_mean=ctrl_mean,
    )

In [4]:
go = pd.read_csv("data/go_v1.csv", index_col=0)

pca_256 = PCA(n_components=256, random_state=0)
go_pca = pca_256.fit_transform(go)
go_pca = pd.DataFrame(go_pca, index=go.index)

## Adamson

In [5]:
pert_data_adamson = PertData("./data/")
pert_data_adamson.load(data_name='adamson')
pert_data_adamson.prepare_split(split="simulation", seed=1)

Found local copy...
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:22
Done!


### Raw target

In [6]:
adamson_data = create_bulk_data(
    pert_data=pert_data_adamson,
    go_pca_=go_pca,
    is_norman=False,
    de_target=False,
)

Number of training samples:  64
Number of test samples:  22


In [7]:
rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=True)
rf.fit(adamson_data['train_x'], adamson_data['train_y'])
adamson_pred = rf.predict(adamson_data['test_x'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    7.5s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 300 out of 300 | elapsed:    0.1s finished


In [8]:
adamson_res = eval(pred=adamson_pred, data=adamson_data, de_target=False)

Mean correlation: 0.7585744776781169


In [9]:
en = ElasticNet(alpha=1, l1_ratio=0.5)
en.fit(adamson_data['train_x'], adamson_data['train_y'])
adamson_pred_en = en.predict(adamson_data['test_x'])

adamson_res_en = eval(pred=adamson_pred_en, data=adamson_data, de_target=False) 

Mean correlation: 0.7083887968406702


### DE target

In [10]:
adamson_data_de = create_bulk_data(
    pert_data=pert_data_adamson,
    go_pca_=go_pca,
    is_norman=False,
    de_target=True,
)

Number of training samples:  64
Number of test samples:  22


In [12]:
np.testing.assert_array_equal(adamson_data['train_x'], adamson_data_de['train_x'])
np.testing.assert_array_equal(adamson_data['test_x'], adamson_data_de['test_x'])

In [16]:
rf_de = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=True)
rf_de.fit(adamson_data_de['train_x'], adamson_data_de['train_y'])
adamson_pred_de = rf_de.predict(adamson_data_de['test_x'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    7.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 300 out of 300 | elapsed:    0.1s finished


In [17]:
adamson_res_de = eval(pred=adamson_pred_de, data=adamson_data_de, de_target=True)

Mean correlation: 0.748541470661813


In [18]:
en = ElasticNet(alpha=1, l1_ratio=0.5)
en.fit(adamson_data_de['train_x'], adamson_data_de['train_y'])
adamson_pred_de = en.predict(adamson_data_de['test_x'])

adamson_res_en_de = eval(pred=adamson_pred_de, data=adamson_data_de, de_target=True)

Mean correlation: 0.7083887968629446


## Norman

In [19]:
pert_data_norman = PertData("./data/")
pert_data_norman.load(data_path='data/norman/')
pert_data_norman.prepare_split(split="simulation", seed=1)

Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:9
combo_seen1:52
combo_seen2:18
unseen_single:37
Done!


### Raw target

In [20]:
norman_data = create_bulk_data(
    pert_data=pert_data_norman,
    go_pca_=go_pca,
    is_norman=True,
    de_target=False,
)

Number of training samples:  167
Number of test samples:  116


In [21]:
rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=True)
rf.fit(norman_data['train_x'], norman_data['train_y'])
norman_pred = rf.predict(norman_data['test_x'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   26.4s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 300 out of 300 | elapsed:    0.1s finished


In [22]:
norman_res = eval(pred=norman_pred, data=norman_data, de_target=False)

Mean correlation: 0.5887369891329108


In [23]:
en = ElasticNet(alpha=1, l1_ratio=0.5)
en.fit(norman_data['train_x'], norman_data['train_y'])
norman_pred_en = en.predict(norman_data['test_x'])

norman_res_en = eval(pred=norman_pred_en, data=norman_data, de_target=False)

Mean correlation: 0.5627775263133756


### DE target

In [24]:
norman_data_de = create_bulk_data(
    pert_data=pert_data_norman,
    go_pca_=go_pca,
    is_norman=True,
    de_target=True,
)

Number of training samples:  167
Number of test samples:  116


In [25]:
rf_de = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=True)
rf_de.fit(norman_data_de['train_x'], norman_data_de['train_y'])
norman_pred_de = rf_de.predict(norman_data_de['test_x'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   26.4s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 300 out of 300 | elapsed:    0.2s finished


In [26]:
norman_res_de = eval(pred=norman_pred_de, data=norman_data_de, de_target=True)

Mean correlation: 0.5926644181130665


In [27]:
en = ElasticNet(alpha=1, l1_ratio=0.5)
en.fit(norman_data_de['train_x'], norman_data_de['train_y'])
norman_pred_de = en.predict(norman_data_de['test_x'])

norman_res_en_de = eval(pred=norman_pred_de, data=norman_data_de, de_target=True)

Mean correlation: 0.5627775263789643


## Replogle

In [28]:
pert_data_replogle = PertData("../data/")
pert_data_replogle.load(data_path='data/replogle_k562_essential/')
pert_data_replogle.prepare_split(split="simulation", seed=1)

Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:273
Done!


### Raw target

In [29]:
replogle_data = create_bulk_data(
    pert_data=pert_data_replogle,
    go_pca_=go_pca,
    is_norman=False,
    de_target=False,
)

Number of training samples:  819
Number of test samples:  273


In [30]:
rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=True)
rf.fit(replogle_data['train_x'], replogle_data['train_y'])
replogle_pred = rf.predict(replogle_data['test_x'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.9min finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 300 out of 300 | elapsed:    0.4s finished


In [31]:
replogle_res = eval(pred=replogle_pred, data=replogle_data, de_target=False)

Mean correlation: 0.49883955611493164


In [32]:
en = ElasticNet(alpha=1, l1_ratio=0.5)
en.fit(replogle_data['train_x'], replogle_data['train_y'])
replogle_pred_en = en.predict(replogle_data['test_x'])

replogle_res_en = eval(pred=replogle_pred_en, data=replogle_data, de_target=False)

Mean correlation: 0.3732669244932551


## DE target

In [34]:
replogle_data_de = create_bulk_data(
    pert_data=pert_data_replogle,
    go_pca_=go_pca,
    is_norman=False,
    de_target=True,
)

Number of training samples:  819
Number of test samples:  273


In [35]:
rf_de = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=True)
rf_de.fit(replogle_data_de['train_x'], replogle_data_de['train_y'])
replogle_pred_de = rf_de.predict(replogle_data_de['test_x'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.9min finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 300 out of 300 | elapsed:    0.4s finished


In [36]:
replogle_res_de = eval(pred=replogle_pred_de, data=replogle_data_de, de_target=True)

Mean correlation: 0.499104525175305


In [37]:
en = ElasticNet(alpha=1, l1_ratio=0.5)
en.fit(replogle_data_de['train_x'], replogle_data_de['train_y'])
replogle_pred_de = en.predict(replogle_data_de['test_x'])

replogle_res_en_de = eval(pred=replogle_pred_de, data=replogle_data_de, de_target=True)

Mean correlation: 0.3732669244862345
